In [ ]:
# neuron to check + generative label
# This is the part BLIP feeds into experiment 3
comp_words = {1:['tench','goldfish', 'ax' 'great white shark', 'yellow', 'natures beauty']}

# filter out vague words
for neuron_id in comp_words:
    for i in range(len(comp_words[neuron_id])):
        comp_words[neuron_id][i] = filter_word(comp_words[neuron_id][i])

In [ ]:
os.chdir(home_dir)
os.chdir('CLIP-dissect')

# del image_set
torch.cuda.empty_cache()

# Get D_probe
pil_data = data_utils.get_data(d_probe)
d_probe_len = len(pil_data)

# Define concept set
words = []

# Get directory of new saved activations
save_names = utils.get_save_names(clip_name = clip_name, target_name = target_name,
                              target_layer = target_layer, d_probe = d_probe,
                              concept_set = concept_set, pool_mode=pool_mode,
                              save_dir = save_dir, newSet = True)
new_target_save_name, new_clip_save_name, new_text_save_name = save_names

# Ensure all previous files are deleted
location = location = ""
if os.path.exists(new_target_save_name):
    target_path = os.path.join(location, new_target_save_name) 
    os.remove(target_path)
if os.path.exists(new_clip_save_name):
    clip_path = os.path.join(location, new_clip_save_name)
    os.remove(clip_path)
if os.path.exists(new_text_save_name):
    text_path = os.path.join(location, new_text_save_name)
    os.remove(text_path)
print('Removed files')

# Neurons to check
neurons_to_check = [i for i in comp_words]

# Block configuration = (# labels to collect, #image per label, (#scoring model, hyperparameter if required))
it_settings = [(15, 10, ('topk-sq-mean', 5))]
# it_settings = [(15, 10, ('topk-sq-mean', 5)), (10, 8, ('topk-sq-mean', 5)), (3, 15, ('topk-sq-mean', 3))]

# Main code
for list_id, orig_id in enumerate(neurons_to_check):
    
    # Add the generative word to the concept set
    words = comp_words[orig_id]
    
    save_names = utils.get_save_names(clip_name = clip_name, target_name = target_name,
                              target_layer = target_layer, d_probe = d_probe,
                              concept_set = concept_set, pool_mode=pool_mode,
                              save_dir = save_dir)
    target_save_name, clip_save_name, text_save_name = save_names
    
    # Make sure saved text files from previous runs are deleted
    if os.path.exists(text_save_name):
        location = location = ""
        text_path = os.path.join(location, text_save_name)
        os.remove(text_path)
        print('Removed files')

    # Save new concept set
    clip_model, clip_preprocess = clip.load(clip_name, device=device)
    text = clip.tokenize(["{}".format(word) for word in words]).to(device)
    utils.save_clip_text_features(clip_model, text, text_save_name, batch_size)

    # Get similarities and target_feats
    similarities, orig_target_feats = utils.get_similarity_from_activations(target_save_name, clip_save_name,
                                                             text_save_name, similarity_fn, device=device)
    
    # Sort labels by similarity highest -> lowest
    _, ids = torch.sort(similarities[orig_id], descending = True)
    
    # Initialize starting concepts
    word_list = []
    for label_id in range(min(it_settings[0][0], len(ids))):
        word_list.append(words[int(ids[label_id])])
    
    print("Neuron {}".format(orig_id))
    
    best_label = ""

    # For each block
    for it_num, it in enumerate(it_settings):

        # Block iteration
        print("Iteration: {}".format(it_num))
        
        # Get block settings
        labels_to_check, num_images_per_prompt, mode_description = it
        mode, hyp_param = mode_description
        
        # Account for added generative label (if necessary)
        labels_to_check = min(labels_to_check, len(word_list))
        
        print(labels_to_check, num_images_per_prompt)
        
        add_im = {}
        add_im_id = {}
        labels = {}
        
        print('Gathering images...', end = "")

        # Generate images for each label
        for label_id in range(labels_to_check):
            pred_label = word_list[label_id]
            labels[label_id] = pred_label # maps label_id to label

            add_im_id[label_id] = [] # initialize image list

            # Generate images
            image_set = pipe(pred_label, generator = generator, num_images_per_prompt = num_images_per_prompt, num_inference_steps=15)

            # Use this if using pre-generated images
            # image_set = get_images(pred_label, num_images_per_prompt, old_path = os.getcwd(), home_dir = home_dir, new_path = '/expanse/lustre/scratch/nbai/temp_project/generated_images')

            for i in range(num_images_per_prompt):
                # Use this if using pre-generated images
                #image = image_set[i]

                # Rescale image
                image = image_set.images[i]
                image = image.resize([32,32])

                new_idx = len(add_im)
                add_im[new_idx] = image # Add image to list
                add_im_id[label_id].append(new_idx) # map new image indices to corresponding label_id
        print('Done')
        del image_set
        torch.cuda.empty_cache()

        # save the new concept set and d_probe
        # reuse cifar100 class to store information (because i'm lazy lol)
        utils.save_new_activations(clip_name = clip_name, target_name = target_name, target_layers = [target_layer],
                          d_probe = 'cifar100_train', new_images = add_im,
                          concept_set = concept_set, wordList = word_list, batch_size = batch_size,
                          device = device, pool_mode=pool_mode, save_dir = save_dir)

        # Get new similarity and target_feats
        save_names = utils.get_save_names(clip_name = clip_name, target_name = target_name,
                                    target_layer = target_layer, d_probe = 'cifar100_train',
                                    concept_set = concept_set, pool_mode=pool_mode,
                                    save_dir = save_dir, newSet = True)

        new_target_save_name, new_clip_save_name, new_text_save_name = save_names   

        similarity, target_feats = utils.get_similarity_from_activations(new_target_save_name, new_clip_save_name,
                                                                 new_text_save_name, similarity_fn, k=len(add_im), device=device)
        
        # Sort images based on activation
        top_vals, top_ids = torch.sort(target_feats, dim=0, descending = True)
        top_image_id = top_ids[:,orig_id]

        # Ranks: label_id -> (indicies of corresponding images in sorted target_feats)
        ranks = {label_id:[] for label_id in range(labels_to_check)}

        # Insert indices of image activations into ranks
        for label_id in range(labels_to_check):
            for i, img_id in enumerate(top_image_id):
                if img_id in add_im_id[label_id]:
                    ranks[label_id].append(i)
            ranks[label_id].sort()
        
        # Reset word_list
        word_list = []
        top_avg = []
        
        # Score labels based on ranks of corresponding generated images
        if mode != 'soft-wpmi':
            top_avg = get_score(ranks, mode, hyp_param, rm_high_outliers = True)
        else:
            new_val, new_id = torch.topk(similarity[orig_id], k=labels_to_check, largest=True)
            print(new_id.shape)
            for i in range(len(new_val)):
                top_avg.append((int(new_val[i]), int(new_id[i])))
            
        if it_num < len(it_settings) - 1: # Generate concept set for next block iteration
            for next_word in range(it_settings[it_num + 1][0]):
                word_list.append(labels[top_avg[next_word][1]])
            print("new list size: {}".format(len(word_list)))
        else: # Record best label
            best_label = labels[top_avg[0][1]]
            
        # Get position of generative label
        in_list = False
        for i in range(len(top_avg)):
            if labels[top_avg[i][1]] == comp_words[orig_id]:
                print("Generative Label found at position: {}".format(i))
                in_list = True
                break
        
        # For debugging purposes
        # if in_list == False:
        #     print("Not found in top {}".format(it_settings[0][0]))
        
        #for i in range(3):
        #   print('Rank {} ({}): {}'.format(i, labels[top_avg[i][1]], ranks[top_avg[i][1]]))
        
        # Remove files for next iteration
        location = ""
        target_path = os.path.join(location, new_target_save_name)  
        clip_path = os.path.join(location, new_clip_save_name)
        text_path = os.path.join(location, new_text_save_name)
        os.remove(target_path)
        os.remove(clip_path)
        os.remove(text_path)
    
    # Print results
    print('------------------------------\n')
    print('Neuron {}:'.format(orig_id))
    print('New Label: {}'.format(best_label))
    print('\n------------------------------')
